In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=182868b9c965fe0474862f037704483219ae97a478feee400eeb70504dd15de3
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [2]:
import numpy as np
import pandas as pd

from string import digits
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from tqdm import tqdm, notebook

from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/kaggle/input/youtube-videos-title-description-comments/GBvideos.csv
/kaggle/input/youtube-videos-title-description-comments/GBtext-details.csv
/kaggle/input/youtube-videos-title-description-comments/UScomments.csv
/kaggle/input/youtube-videos-title-description-comments/GB_category_id.json
/kaggle/input/youtube-videos-title-description-comments/US_category_id.json
/kaggle/input/youtube-videos-title-description-comments/UStext-details.csv
/kaggle/input/youtube-videos-title-description-comments/GBcomments.csv
/kaggle/input/youtube-videos-title-description-comments/USvideos.csv


### Reading Data

Reading GB data

In [31]:
gb_detail_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/GBvideos.csv', on_bad_lines='skip')
gb_detail_df.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,28,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240,13548,705,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,13.09
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,This Morning,24,this morning|interview|holly willoughby|philli...,494203,2651,1309,0,https://i.ytimg.com/vi/AqokkXoa7uE/default.jpg,13.09
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,emmablackery,24,emmablackery|emma blackery|emma|blackery|briti...,142819,13119,151,1141,https://i.ytimg.com/vi/YPVcg45W0z4/default.jpg,13.09
3,T_PuZBdT2iM,getting into a conversation in a language you ...,ProZD,1,skit|korean|language|conversation|esl|japanese...,1580028,65729,1529,3598,https://i.ytimg.com/vi/T_PuZBdT2iM/default.jpg,13.09
4,NsjsmgmbCfc,Baby Name Challenge!,Sprinkleofglitter,26,sprinkleofglitter|sprinkle of glitter|baby gli...,40592,5019,57,490,https://i.ytimg.com/vi/NsjsmgmbCfc/default.jpg,13.09


In [11]:
gb_text_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/GBtext-details.csv')
gb_text_df = gb_text_df.drop('Unnamed: 0', axis=1)
gb_text_df.head()

,id,title,description
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Live Apple Event - Apple September Event 2017 ...
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,Subscribe now for more! http://bit.ly/1JM41yF\...
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,extra vlogs: http://youtube.com/c/vloggery \ni...
3,T_PuZBdT2iM,getting into a conversation in a language you ...,My Twitter: https://twitter.com/prozd\nMy Let'...
4,NsjsmgmbCfc,Crazy Baby Names! | MOTHERHOOD,"All things Names! Cute, weird, wonderful baby ..."


In [5]:
gb_comments_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/GBcomments.csv', on_bad_lines='skip')
gb_comments_df.head()

,video_id,comment_text,likes,replies
0,jt2OHQh0HoQ,It's more accurate to call it the M+ (1000) be...,0,0
1,jt2OHQh0HoQ,To be there with a samsung phone\n😂😂😂,1,0
2,jt2OHQh0HoQ,"Thank gosh, a place I can watch it without hav...",0,0
3,jt2OHQh0HoQ,What happened to the home button on the iPhone...,0,0
4,jt2OHQh0HoQ,Power is the disease. Care is the cure. Keep...,0,0


Reading us data

In [32]:
us_detail_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/USvideos.csv', on_bad_lines='skip')
us_detail_df.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


In [12]:
us_text_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/UStext-details.csv')
us_text_df = us_text_df.drop('Unnamed: 0', axis=1)
us_text_df.head()

,id,title,description
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Grab your 365 DAY MAVERICK MERCH! Exclusively ...
1,cLdxuaxaQwc,My Response,NaN
2,WYYvHb03Eog,Apple iPhone X first look,The Apple iPhone X is the new super flagship p...
3,sjlHnJvXdQs,iPhone X (parody),MORE APPLE STUFF ► https://www.youtube.com/wat...
4,cMKX2tE5Luk,The Disaster Artist | Official Trailer HD | A24,SUBSCRIBE: http://bit.ly/A24subscribe\n\nBased...


Concatenating the GB and US data

In [18]:
us_comments_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/UScomments.csv', on_bad_lines='skip')
us_comments_df.head()

/tmp/ipykernel_31/3472227992.py:1: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  us_comments_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/UScomments.csv', on_bad_lines='skip')


,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0
4,XpVt6Z1Gjjo,trending 😉,3,0


In [16]:
text_df = pd.concat([gb_text_df, us_text_df], axis=0)
print("Len of GB data - ", len(gb_text_df))
print("Len of GB data - ", len(us_text_df))
print("Len of combined data - ", len(text_df))
text_df.head()

Len of GB data -  7120
Len of GB data -  7227
Len of combined data -  14347


,id,title,description
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Live Apple Event - Apple September Event 2017 ...
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,Subscribe now for more! http://bit.ly/1JM41yF\...
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,extra vlogs: http://youtube.com/c/vloggery \ni...
3,T_PuZBdT2iM,getting into a conversation in a language you ...,My Twitter: https://twitter.com/prozd\nMy Let'...
4,NsjsmgmbCfc,Crazy Baby Names! | MOTHERHOOD,"All things Names! Cute, weird, wonderful baby ..."


In [19]:
comments_df = pd.concat([gb_comments_df, us_comments_df], axis=0)
print("Len of GB data - ", len(gb_comments_df))
print("Len of GB data - ", len(us_comments_df))
print("Len of combined data - ", len(comments_df))
comments_df.head()

Len of GB data -  718452
Len of GB data -  691400
Len of combined data -  1409852


,video_id,comment_text,likes,replies
0,jt2OHQh0HoQ,It's more accurate to call it the M+ (1000) be...,0,0
1,jt2OHQh0HoQ,To be there with a samsung phone\n😂😂😂,1,0
2,jt2OHQh0HoQ,"Thank gosh, a place I can watch it without hav...",0,0
3,jt2OHQh0HoQ,What happened to the home button on the iPhone...,0,0
4,jt2OHQh0HoQ,Power is the disease. Care is the cure. Keep...,0,0


In [33]:
details_df = pd.concat([gb_detail_df, us_detail_df], axis=0)
print("Len of GB data - ", len(gb_detail_df))
print("Len of GB data - ", len(us_detail_df))
print("Len of combined data - ", len(details_df))
details_df.head()

Len of GB data -  7993
Len of GB data -  7992
Len of combined data -  15985


,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,28,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240,13548,705,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,13.09
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,This Morning,24,this morning|interview|holly willoughby|philli...,494203,2651,1309,0,https://i.ytimg.com/vi/AqokkXoa7uE/default.jpg,13.09
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,emmablackery,24,emmablackery|emma blackery|emma|blackery|briti...,142819,13119,151,1141,https://i.ytimg.com/vi/YPVcg45W0z4/default.jpg,13.09
3,T_PuZBdT2iM,getting into a conversation in a language you ...,ProZD,1,skit|korean|language|conversation|esl|japanese...,1580028,65729,1529,3598,https://i.ytimg.com/vi/T_PuZBdT2iM/default.jpg,13.09
4,NsjsmgmbCfc,Baby Name Challenge!,Sprinkleofglitter,26,sprinkleofglitter|sprinkle of glitter|baby gli...,40592,5019,57,490,https://i.ytimg.com/vi/NsjsmgmbCfc/default.jpg,13.09


In [20]:
device = 'cuda'
if torch.cuda.is_available():      
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


Looking at data for a particular video

In [21]:
text_df.rename(columns = {'id':'video_id'}, inplace = True)

In [22]:
text_df['video_id'].sample(10)

1823    chfg6YYiQpU
2413    1yYV9-KoSUM
5356    1hvdr7RjNRk
3614    bd7IGZ6dulM
2895    0AaApxaD170
2176    7ocf8suHnP0
6519    Q0CbN8sfihY
2122    RaYqX-0AwRw
1704    74zJ4scJzNs
846     cMKX2tE5Luk
Name: video_id, dtype: object

Title

In [24]:
text_df.iloc[2176].title

"ULTIMATE GYMNASTICS FAILS | Who's in the Worlds Team?"

Description

In [25]:
text_df.iloc[2176].description

"➢ Join my FREE 7 Steps to Success - http://www.bodybible.life/\n➢ Merch- http://www.nwclothing.co.uk/\n➢ Enquiries - luke@activatemanagement.co.uk\n\n➢ Handstand Programme 🤸🏼\u200d♂️https://bodybible.life/individual-programmes-2/\n➢ Depop -https://www.depop.com/joannaj\n➢ Gymnastics Gear ~ https://www.turn-gymnastics.com/\n➢ Intro Song ~ Tom Zanetti, Darlin'\n➢ Soundcloud ~ https://soundcloud.com/nile-wilson-2/sets/wobblers\n➢Instagram https://instagram.com/nilemw/\n➢ Body Bible instagram https://www.instagram.com/nw_bodybible/\n➢Twitter https://twitter.com/NileMW\n➢Facebook https://www.facebook.com/NileWilsonGy…\n➢SnapChat - nilewilson\n➢Songs ~ https://www.youtube.com/user/officialdjwelshy"

Comments

In [35]:
video_comments = comments_df[comments_df['video_id'] == '7ocf8suHnP0'].comment_text.values
print("Total comments for video - 7ocf8suHnP0 = ", len(video_comments))
print("Channel title for video - 7ocf8suHnP0 = ", details_df[details_df['video_id'] == '7ocf8suHnP0'].channel_title)
video_comments[:5]

Total comments for video - 7ocf8suHnP0 =  600
Channel title for video - 7ocf8suHnP0 =  2030    Nile Wilson
2244    Nile Wilson
2450    Nile Wilson
2663    Nile Wilson
2935    Nile Wilson
3181    Nile Wilson
Name: channel_title, dtype: object


array(["James x'D especially when he gives up in the first clip hahah, but gottta admit Danny's fail is hard to\xa0top loll, poor him!!",
       'James for the fail / win made me 😂 and congrats for making the team keep up the hard work',
       "The second one hitting the bars in the worst place possible 😂 nile's videos are my daily dose of good looking, charming men that are painfully absent in my own life",
       'Nile, cringiest: Your fall on the high bar.  It echoed thru the building.  Funniest:  the Purvis, definitely! Love the video!',
       'Interesting.. I always wondered what those hand guard things were.'],
      dtype=object)

In [36]:
def clean_text(text, remove_stopwords=True):
    text = text.lower()
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words('english'))
        text = [w for w in text if w not in stops]
        text = ' '.join(text)
        
    return text

In [37]:
def clean_data(data):
    cleaned_data = []
    for doc in notebook.tqdm(data):
        text = clean_text(doc, False)
        cleaned_data.append(text)
    return cleaned_data

In [39]:
clean_data([text_df.iloc[2176].description])

  0%|          | 0/1 [00:00<?, ?it/s]

['➢ join my free 7 steps to success   ➢ merch  ➢ enquiries   luke activatemanagement co uk\n\n➢ handstand programme 🤸🏼\u200d♂️➢ depop  ➢ gymnastics gear ~ ➢ intro song ~ tom zanetti  darlin \n➢ soundcloud ~ ➢instagram ➢ body bible instagram ➢twitter ➢facebook ➢snapchat   nilewilson\n➢songs ~ ']

In [46]:
from nltk.tokenize import RegexpTokenizer
import re
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    return " ".join(filtered_words)

preprocess(text_df.iloc[2176].description)

'join free steps success merch enquiries luke activatemanagement handstand programme depop gymnastics gear intro song tom zanetti darlin soundcloud instagram body bible instagram twitter facebook snapchat nilewilson songs'

In [47]:
embedder = SentenceTransformer('all-MiniLM-L6-v2', device=device)
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device=device)

In [53]:
cleaned_description = preprocess(text_df.iloc[2176].description)
title = text_df.iloc[2176].title
channel_title = details_df[details_df['video_id'] == '7ocf8suHnP0'].channel_title.values[0]
query_text = title + '. ' + cleaned_description + '. ' + channel_title
query_text

"ULTIMATE GYMNASTICS FAILS | Who's in the Worlds Team?. join free steps success merch enquiries luke activatemanagement handstand programme depop gymnastics gear intro song tom zanetti darlin soundcloud instagram body bible instagram twitter facebook snapchat nilewilson songs. Nile Wilson"

In [54]:
comments = comments_df[comments_df['video_id'] == '7ocf8suHnP0'].comment_text.values
comments[0]

"James x'D especially when he gives up in the first clip hahah, but gottta admit Danny's fail is hard to\xa0top loll, poor him!!"

In [55]:
def get_embeddings(data):
    return embedder.encode(data, convert_to_tensor=True)

In [58]:
comments_embeddings = get_embeddings(comments)
comments_embeddings.shape

Batches:   0%|          | 0/19 [00:00<?, ?it/s]

torch.Size([600, 384])

In [59]:
query_embedding = get_embeddings([query_text])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [61]:
hits = util.semantic_search(query_embedding, comments_embeddings, top_k=100)
hits = hits[0]

In [62]:
hits[:10]

[{'corpus_id': 108, 'score': 0.5700269937515259},
 {'corpus_id': 214, 'score': 0.5700269937515259},
 {'corpus_id': 318, 'score': 0.5700269937515259},
 {'corpus_id': 423, 'score': 0.5700269937515259},
 {'corpus_id': 523, 'score': 0.5700269937515259},
 {'corpus_id': 100, 'score': 0.49845829606056213},
 {'corpus_id': 210, 'score': 0.49845829606056213},
 {'corpus_id': 314, 'score': 0.49845829606056213},
 {'corpus_id': 419, 'score': 0.49845829606056213},
 {'corpus_id': 519, 'score': 0.49845829606056213}]

In [63]:
comments[108]

'What happened to the ultimate gymnastics challenge??'

In [64]:
comments[214]

'What happened to the ultimate gymnastics challenge??'

In [65]:
comments[318]

'What happened to the ultimate gymnastics challenge??'

In [66]:
comments[519]

'you should make a video of trying girls gymnastics'

In [68]:
hits[99]

{'corpus_id': 360, 'score': 0.28074726462364197}

In [69]:
comments[360]

'Love you Nile'

In [70]:
all_comments_score = util.semantic_search(query_embedding, comments_embeddings, top_k=600)
all_comments_score = all_comments_score[0]

In [71]:
all_comments_score[599]

{'corpus_id': 530, 'score': -0.023314600810408592}

In [72]:
comments[530]

'I just go harassed by kids at a park'

In [77]:
all_comments_score[555]

{'corpus_id': 114, 'score': 0.04093947261571884}

In [78]:
comments[114]

'6:01'